In [10]:
import asyncio
import pandas as pd
import plotly.graph_objs as go
import plotly.express as px

import warnings
warnings.filterwarnings("ignore")


#df_c4z = pd.read_csv("order_test_fet_15_11_c4z.csv")
df_26nf = pd.read_csv("csv/test.csv")
#data_c4z=data_c4z[:1000]
#data_26nf=data_26nf[:1000]

#df_index_price = pd.read_csv("2024-11-14.csv")
#df_index_price=df_index_price[:1000]


In [11]:
# #df_c4z.drop(columns=['marketId', 'subaccountId', 'executionType', 'orderHash','quantity'], inplace=True)
df_26nf.drop(columns=['marketId', 'subaccountId', 'executionType', 'orderHash'], inplace=True)


#df_26nf['price'] = df_26nf['price'] * 10**12
#df_c4z['price'] = df_c4z['price'] * 10**12

In [12]:
df_26nf

,orderType,price,quantity,updated_at,state,created_at
0,buy,15.559,2099000000,2025-01-22 17:25:21.195,canceled,2025-01-22 17:25:06.954
1,sell,15.602,2099000000,2025-01-22 17:25:21.195,canceled,2025-01-22 17:25:06.954
2,buy,15.549,2101000000,2025-01-22 17:25:21.195,booked,2025-01-22 17:25:21.195
3,sell,15.593,2101000000,2025-01-22 17:25:21.195,booked,2025-01-22 17:25:21.195
4,buy,15.566,2098000000,2025-01-22 17:25:06.954,canceled,2025-01-22 17:25:04.446
...,...,...,...,...,...,...
2713,sell,15.486,2115000000,2025-01-22 16:25:30.380,canceled,2025-01-22 16:25:27.442
2714,buy,15.447,2115000000,2025-01-22 16:25:27.442,canceled,2025-01-22 16:25:25.506
2715,sell,15.490,2115000000,2025-01-22 16:25:27.442,canceled,2025-01-22 16:25:25.506
2716,buy,15.442,2115000000,2025-01-22 16:25:25.506,canceled,2025-01-22 16:25:24.849


In [13]:
df_transformed = df_26nf.pivot_table(
    index=["updated_at", "state", "created_at"],
    columns="orderType",
    values="price",
    aggfunc="first"
).reset_index()

print(df_transformed.columns) 
# Rename columns for clarity
#df_transformed.columns = ["updated_at", "state", "created_at", "buy_price", "sell_price"]
df_transformed["buy"]= df_transformed["buy"]/1e6
df_transformed["sell"]= df_transformed["sell"]/1e6
#df_transformed["buy_po"]= df_transformed["buy_po"]/1e6
#df_transformed["sell_po"]= df_transformed["sell_po"]/1e6
# Display the transformed DataFrame
df_transformed["offset"] = 100 * (df_transformed["sell"]-df_transformed["buy"])/ ((df_transformed["sell"]+df_transformed["buy"])/2)/2
df_transformed

Index(['updated_at', 'state', 'created_at', 'buy', 'sell'], dtype='object', name='orderType')


orderType,updated_at,state,created_at,buy,sell,offset
0,2025-01-22 16:25:25.506,canceled,2025-01-22 16:25:24.849,0.000015,0.000015,0.135808
1,2025-01-22 16:25:27.442,canceled,2025-01-22 16:25:25.506,0.000015,0.000015,0.138992
2,2025-01-22 16:25:30.380,canceled,2025-01-22 16:25:27.442,0.000015,0.000015,0.139028
3,2025-01-22 16:25:32.374,canceled,2025-01-22 16:25:30.380,0.000015,0.000015,0.139001
4,2025-01-22 16:25:35.714,canceled,2025-01-22 16:25:32.374,0.000015,0.000015,0.142202
...,...,...,...,...,...,...
1354,2025-01-22 17:25:00.294,canceled,2025-01-22 17:24:59.597,0.000016,0.000016,0.137931
1355,2025-01-22 17:25:04.446,canceled,2025-01-22 17:25:00.294,0.000016,0.000016,0.137896
1356,2025-01-22 17:25:06.954,canceled,2025-01-22 17:25:04.446,0.000016,0.000016,0.141134
1357,2025-01-22 17:25:21.195,booked,2025-01-22 17:25:21.195,0.000016,0.000016,0.141288


In [14]:
import plotly.graph_objects as go


fig = go.Figure()

# Add traces for each y-column
fig.add_trace(go.Scatter(x=df_transformed['updated_at'], y=df_transformed['buy'],
                         mode='lines', name='buy '))
fig.add_trace(go.Scatter(x=df_transformed['updated_at'], y=df_transformed['sell'],
                         mode='lines', name='sell '))
#fig.add_trace(go.Scatter(x=df_transformed['updated_at'], y=df_transformed['buy_po'],
#                         mode='lines', name='buy '))
#fig.add_trace(go.Scatter(x=df_transformed['updated_at'], y=df_transformed['sell_po'],
#                         mode='lines', name='sell '))
# Update layout for titles and axes
fig.update_layout(
    title="Offsets over Time",
    xaxis_title="Updated At",
    yaxis_title="Offset Value",
    legend_title="Offsets",
)

# Show the figure
fig.show()


In [15]:
import plotly.express as px

# Create a sample DataFrame

# Create a line plot from the DataFrame
fig = px.line(df_transformed, x='updated_at', y='offset', title='Offset from c4z order')

# Show the plot
fig.show()
